![Digital Futures](https://github.com/digital-futures-academy/DataScienceMasterResources/blob/main/Resources/datascience-notebook-header.png?raw=true)

# Setting up the Project Environment

Ultimately, the Python code written here will be extracted to scripts for execution in an automated pipeline.  To facilitate this, there is a need to set up a project environment that will allow for the execution of the code in a controlled and reproducible environment.

In the initial stages of the activities, the packages needed are `requests` and `pytest`.  The `requests` package is used to make HTTP requests to the API, while `pytest` is used for testing the code we also need *BeautifulSoup* (package name `beautifulsoup4`.  In later activities, you may need to install additional packages.  To do this, add the packages to the `pip install` command below and re-run the cell.

> **Remember:** The goal is to create a set of code cells that can be extracted to separate scripts for execution in an automated pipeline.  Therefore, the code should be kept in 3 distinct cells:
> 
> - **Shell Commands**:  Used to set up the project environment
> 
> - **Python Tests**: Used to test the Python production scripts both now and as part of the automated pipeline
> 
> - **Python Production Code**: The Python code that will be extracted to a script to execute during the pipeline

---

# Environment Setup Scripts

If you are running this notebook after cloning and have not set up your environment to run shell commands, you will need to run the following commands in your terminal to set up the environment.

> **NOTE:**  These commands need to be executed in the terminal.  
>
> Open a terminal at the root of your project before executing these commands
> 
> Until your environment is set up, Jupyter Notebooks will not be able to run **shell** scripts.

```sh
# Create a virtual environment (add the command below)
python3 -m venv .venv # Note: This command could also be python -m venv .venv # python3 and python are a symlink to the python version installed on your system

# Activate the virtual environment 
source .venv/bin/activate

# Install required package to execute shell commands from Jupyter Notebook
pip install ipykernel               ## OR 
pip install -r requirements.txt     ## IF there is already a requirements.txt file CONTAINING ipykenrnel in the project
```


In [7]:
# Install the necessary packages
!pip install requests pytest beautifulsoup4

# Create a requirements.txt file
!pip freeze > requirements.txt

> **Note:** 
> The `!` at the beginning of the lines is a special character in Jupyter Notebooks that allows you to run shell commands from the notebook.  
> These will need to be removed from any commands that are to be exported to a `.sh` shell script file for the pipeline.

---

# Python Tests

Develop any tests for functions in separate cells below.  The first has been provided for you as an example, add others as necessary.

In [8]:
# Test request_to_scrape
from unittest import mock
import ipytest

def test_request_to_scrape():
    # Arrange
    example_url = "https://www.example.com"
    with mock.patch('requests.get') as mock_request:
        mock_response = mock.Mock()
        mock_response.status_code = 200
        mock_response.text = "This is the text of the page"
        mock_response.url = example_url

        mock_request.return_value = mock_response

        # Act
        response = request_to_scrape(example_url)
        
        # Assert
        mock_request.assert_called_once_with(example_url)
        assert response.status_code == 200
        assert response.text == "This is the text of the page"
        assert response.url == example_url
        
        
def test_request_to_scrape_unsuccessful_request():
    # Arrange
    example_url = "https://www.example.com"
    with mock.patch('requests.get') as mock_request:
        mock_response = mock.Mock()
        mock_response.status_code = 404
        mock_response.text = "Not Found"
        mock_response.url = example_url
        mock_request.return_value = mock_response

        # Act
        response = request_to_scrape(example_url)
        
        # Assert
        mock_request.assert_called_once_with(example_url)
        assert response == f"Request failed for URL: {example_url} with status code 404"
        
        
def test_request_to_scrape_exception_handling():
    # Arrange
    example_url = "https://www.example.com"
    with mock.patch('requests.get', side_effect=Exception("Network error")) as mock_request:
        
        # Act
        response = request_to_scrape(example_url)
        
        # Assert
        mock_request.assert_called_once_with(example_url)
        assert response == f"{ERROR_UNEXPECTED}: {example_url} with error Network error"

ipytest.run()


======================================= test session starts =======================================
platform win32 -- Python 3.13.0, pytest-8.3.3, pluggy-1.5.0
rootdir: c:\Users\aqora\dfprojects\python_projects\webscraping\web-scraping-syed-abdi
collected 3 items

t_04b8ec2ae0374c94b993decaa4f66e2c.py ...                                                    [100%]

======================================== 3 passed in 0.07s ========================================


<ExitCode.OK: 0>

In [ ]:
# test extract_book_categories Production Code
def test_extract_book_categories_with_valid_html():
    """
    Test that `extract_book_categories` correctly parses valid HTML input.
    """  # noqa E501
    categories = extract_book_categories(
        CATEGORY_HTML_VALID, 
        TEST_URL
    )
    assert categories == { 
        **TEST_CATEGORIES_WITH_LINKS[0],
        **TEST_CATEGORIES_WITH_LINKS[1]
    }

...F                                                                                         [100%]
============================================ FAILURES =============================================
__________________________ test_extract_book_categories_with_valid_html ___________________________

    def test_extract_book_categories_with_valid_html():
    
        # Arrange
        url = "http://books.toscrape.com/"
    
        mock_response = mock.Mock()
        mock_response.status_code = 200
        mock_response.text = """ <ul class="nav-list">
                <li><a href="catalogue/category/books/travel_2/index.html">Travel</a></li>
                <li><a href="catalogue/category/books/mystery_3/index.html">Mystery</a></li>
            </ul>
        """
        with mock.patch('__main__.request_to_scrape') as mock_request_to_scrape:
    
    
            # Act
    
            response = extract_book_categories(url)
    
            # Assert
    
            mock_request_to_sc

<ExitCode.TESTS_FAILED: 1>

### Run the tests

Run the cell containing the `ipytest.run()` command to execute the tests.  The tests should all fail until you have written the production code.

Don't forget to run the installation and initialisation cell too on the first time you run the tests!


---

# Python Production Code


Develop any functions for use as production code in separate cells below. The first has been provided as an example under the Production Constants, add others as necessary.

### PRODUCTION CONSTANTS

In [9]:
# PRODUCTION CONSTANTS

# Constants for status messages
STATUS_SUCCESS = "success"
STATUS_ERROR = "error"
ERROR_NOT_HTML = "The response is not HTML"
ERROR_REQUEST_FAILED = "Request failed for URL"
ERROR_UNEXPECTED = "Unexpected error for URL"

# HTML Parser
HTML_PARSER = "html.parser"

### `request_to_scrape` Production Code

In [22]:
# request_to_scrape Production Code
import requests

def request_to_scrape(url):
    try:
        response = requests.get(url)
        if response.status_code == 404:
            return f"{ERROR_REQUEST_FAILED}: {url} with status code 404"
        return response
        #return response.status_code
    except Exception as e:
        return f"{ERROR_UNEXPECTED}: {url} with error {e}"
    

---

# Python Execution Code

Develop any code to call the developed functions below.  Add additional cells so you don't need to re-run all of the code when you develop further scripts.

In [ ]:
# Python Execution Code
# extract_book_categories Production Code

import requests
from bs4 import BeautifulSoup

def extract_book_categories(url):
    
    response = request_to_scrape(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, HTML_PARSER)
        
        return soup
        
        
        
        
extract_book_categories('http://books.toscrape.com/')
 
 
 


<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link href="s

---

# Jupyter Notebook Test and Linting Set Up

To run `pytest` scripts in a Jupyter Notebook cell, we need to install the `ipytest` package.  This package is NOT required for a pipeline and therefore it can be removed from the `requirements.txt` file before adding the production code to the pipeline.

To run linting, we need to install 2 packages `nbqa` and `flake8`.  We will make sure that `flake8` is included in the `requirements.txt` file when constructing the pipeline so that we can lint as part of the pipeline tests.

Run the following cell to install the `ipytest`, `nbqa` and `flake8` packages and a coverage package to help determine if all of your production code is executed during the tests!

This cell only needs to be run once (or after restarting the notebook kernel) to set up the environment for testing and linting.


In [12]:
# Install the `ipytest`, `nbqa` and `flake8` packages
!pip install ipytest nbqa flake8

### Set up `ipytest` to execute `pytest` scripts in Jupyter Notebook

In [13]:
# Configure ipytest for Jupyter Notebook

import ipytest
ipytest.autoconfig(rewrite_asserts=True, magics=True)

### Create a *config* file for `flake8`

Run this script to create a file in your project root

In [14]:
# Create a config file and ignore some flake8 rules
!echo "[flake8]" > .flake8
!echo "ignore = E402, W291, F811" >> .flake8

# Execute the tests and linting in the Jupyter Notebook

Run the following cell ***EVERY TIME*** you want to run the tests and linting that you have written in the *Python Tests* cell above.

>**Note:**
>
> This entire section does not need to be part of any pipeline scripts.  
> It is only required for the Jupyter Notebook environment during development.


In [15]:
# Run the tests
ipytest.run("-vv", "-ss")

======================================= test session starts =======================================
platform win32 -- Python 3.13.0, pytest-8.3.3, pluggy-1.5.0 -- c:\Users\aqora\dfprojects\python_projects\webscraping\web-scraping-syed-abdi\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\aqora\dfprojects\python_projects\webscraping\web-scraping-syed-abdi
collecting ... collected 3 items

t_04b8ec2ae0374c94b993decaa4f66e2c.py::test_request_to_scrape PASSED
t_04b8ec2ae0374c94b993decaa4f66e2c.py::test_request_to_scrape_unsuccessful_request PASSED
t_04b8ec2ae0374c94b993decaa4f66e2c.py::test_request_to_scrape_exception_handling PASSED

======================================== 3 passed in 0.14s ========================================


<ExitCode.OK: 0>

## Run the linter

Run this script each time you want to lint your code

In [16]:
# Run the linter
!nbqa flake8 --show-source --format=pylint webscraping.ipynb

webscraping.ipynb:cell_2:2: [E402] module level import not at top of file
from unittest import mock
^
webscraping.ipynb:cell_2:3: [E402] module level import not at top of file
import ipytest
^
webscraping.ipynb:cell_2:19: [W293] blank line contains whitespace
        
^
webscraping.ipynb:cell_2:25: [W293] blank line contains whitespace
        
^
webscraping.ipynb:cell_2:26: [W293] blank line contains whitespace
        
^
webscraping.ipynb:cell_2:39: [W293] blank line contains whitespace
        
^
webscraping.ipynb:cell_2:42: [E501] line too long (88 > 79 characters)
        assert response == f"Request failed for URL: {example_url} with status code 404"
                                                                               ^
webscraping.ipynb:cell_2:43: [W293] blank line contains whitespace
        
^
webscraping.ipynb:cell_2:44: [W293] blank line contains whitespace
        
^
webscraping.ipynb:cell_2:48: [E501] line too long (92 > 79 characters)
    with mock.patch('reques


---
